# EDA (statistics, quality checks)

In [21]:
import pandas as pd
import re

# ── Load data ─────────────────────────────────────────────────────────────────
df = pd.read_csv('../Data/spotify_reviews.csv')


# ── Basic text features ───────────────────────────────────────────────────────
df['text_len']      = df['Content'].str.len()
df['word_count']    = df['Content'].str.split().str.len()
df['has_non_ascii'] = df['Content'].str.contains(r'[^\x00-\x7F]', regex=True)

# ── Descriptive stats ─────────────────────────────────────────────────────────
print("=" * 50)
print("📊 Text Statistics")
print("=" * 50)
print(df[['text_len', 'word_count']].describe().round(2))

# ── Noise ratio ───────────────────────────────────────────────────────────────
print("\n" + "=" * 50)
print("⚠️  Noise / Non-ASCII ratio")
print("=" * 50)
print(f"  Rows with emojis / non-ASCII chars: "
    f"{df['has_non_ascii'].sum()} "
    f"({df['has_non_ascii'].mean()*100:.2f} %)")

# ── Top 10 words ──────────────────────────────────────────────────────────────
print("\n" + "=" * 50)
print("🔤 Top 10 Most Common Words")
print("=" * 50)
top_words = (
    df['Content']
    .str.lower()
    .str.replace(r'[^\w\s]', '', regex=True)   # strip punctuation first
    .str.split()
    .explode()
    .value_counts()
    .head(10)
    .reset_index()
)
top_words.columns = ['word', 'count']
print(top_words.to_string(index=False))

# ── Quirks list ───────────────────────────────────────────────────────────────
print("\n" + "=" * 50)
print("🔍 Quirks / Data Quality Flags")
print("=" * 50)
empty_rows      = df['Content'].isna().sum()
blank_rows      = (df['Content'].str.strip() == '').sum()
duplicate_rows  = df['Content'].duplicated().sum()
very_short      = (df['word_count'] < 3).sum()
very_long       = (df['word_count'] > df['word_count'].quantile(0.99)).sum()

quirks = {
    "Missing / NaN texts"       : empty_rows,
    "Blank texts"               : blank_rows,
    "Duplicate texts"           : duplicate_rows,
    "Very short texts (< 3 w)"  : very_short,
    "Very long texts (> 99 pct)": very_long,
    "Non-ASCII / emoji texts"   : df['has_non_ascii'].sum(),
}
for label, val in quirks.items():
    flag = "🚩" if val > 0 else "✅"
    print(f"  {flag}  {label:<35}: {val}")

# ── 3 raw samples ─────────────────────────────────────────────────────────────
print("\n" + "=" * 50)
print("📝 3 Random Raw Samples")
print("=" * 50)
for i, sample in enumerate(df['Content'].sample(3, random_state=42).tolist(), 1):
    print(f"\n  [{i}] {sample}")

📊 Text Statistics
       text_len  word_count
count    100.00      100.00
mean     117.02       21.30
std      108.00       20.11
min       24.00        3.00
25%       43.00        8.00
50%       78.50       13.50
75%      134.50       24.25
max      500.00      106.00

⚠️  Noise / Non-ASCII ratio
  Rows with emojis / non-ASCII chars: 17 (17.00 %)

🔤 Top 10 Most Common Words
   word  count
     to     74
    the     70
    app     63
premium     48
   this     46
     is     43
    and     35
      i     33
spotify     30
    now     29

🔍 Quirks / Data Quality Flags
  ✅  Missing / NaN texts                : 0
  ✅  Blank texts                        : 0
  ✅  Duplicate texts                    : 0
  ✅  Very short texts (< 3 w)           : 0
  🚩  Very long texts (> 99 pct)         : 1
  🚩  Non-ASCII / emoji texts            : 17

📝 3 Random Raw Samples

  [1] u just make everything paid trash app

  [2] I really love this app this app always give me best jazz to Jinja my self

  [3] Wher

# Punctuation feature extraction

In [22]:
import string

# Extract punctuation intensity BEFORE cleaning
df['exclamation_count'] = df['Content'].str.count(r'!')
df['question_count'] = df['Content'].str.count(r'\?')
df['has_repeated_punct'] = df['Content'].str.contains(r'[!?]{2,}', regex=True)




# Modular preprocessing pipeline

In [23]:
import re
import unicodedata
import contractions
import string

# ── Modular preprocessing function ─────────────────────────────────────────
def preprocess_text(text: str) -> str:
    """
    Comprehensive text preprocessing pipeline.
    Handles: HTML, Unicode, contractions, URLs, social handles, punctuation.
    """
    if not isinstance(text, str):
        return ""
    
    # 1. Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # 2. Fix contractions & lowercase (SKIP ASCII normalization)
    text = contractions.fix(text).lower()
    
    # 3. Remove URLs, handles, hashtags
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'@\w+|#\w+', '', text)
    
    # 4. Keep punctuation and emojis
    
    # 5. Normalize whitespace
    return ' '.join(text.split())

# ── Apply preprocessing ────────────────────────────────────────────────────
df['cleaned'] = df['Content'].apply(preprocess_text)

# ── Preview ───────────────────────────────────────────────────────────────
print("=" * 60)
print("🔍 Preprocessing Pipeline Preview (3 samples)")
print("=" * 60)
for i, (raw, cleaned) in enumerate(zip(df['Content'].head(3), df['cleaned'].head(3)), 1):
    print(f"\n  [{i}] Raw     : {raw[:80]}...")
    print(f"      Cleaned : {cleaned[:80]}...")


# ── Vocabulary check ──────────────────────────────────────────────────────
vocab_size = len(set(' '.join(df['cleaned']).split()))
print(f"\n📊 Vocabulary size: {vocab_size:,} unique words")

# ── Sequence Length Analysis ──────────────────────────────────────────────
df['token_count'] = df['cleaned'].str.split().str.len()

print("\n" + "=" * 50)
print("📏 Sequence Length Stats (post-cleaning)")
print("=" * 50)
print(df['token_count'].describe().round(2))
print(f"\n  Avg tokens/review : {df['token_count'].mean():.1f}")
print(f"  Median            : {df['token_count'].median():.1f}")
print(f"  95th percentile   : {df['token_count'].quantile(0.95):.0f}")
print(f"\n  → Relevant for max_length in transformer models (e.g. BERT = 512 tokens)")

🔍 Preprocessing Pipeline Preview (3 samples)

  [1] Raw     : Spotify is very good, got many songs and suggests new ones but I've to say the a...
      Cleaned : spotify is very good, got many songs and suggests new ones but i have to say the...

  [2] Raw     : Worst update ever!!! Now, you can't even forward or backward lyrics/part in a so...
      Cleaned : worst update ever!!! now, you cannot even forward or backward lyrics/part in a s...

  [3] Raw     : Intolerable it is, as they are now demanding remuneration for the recurring audi...
      Cleaned : intolerable it is, as they are now demanding remuneration for the recurring audi...

📊 Vocabulary size: 719 unique words

📏 Sequence Length Stats (post-cleaning)
count    100.00
mean      21.79
std       20.59
min        3.00
25%        8.75
50%       14.00
75%       24.25
max      110.00
Name: token_count, dtype: float64

  Avg tokens/review : 21.8
  Median            : 14.0
  95th percentile   : 66

  → Relevant for max_length in 

In [24]:
# ── Manual Inspection: Sample 10 Cleaned Records ──────────────────────────
print("=" * 80)
print("🔍 Manual Inspection: Raw vs Cleaned (10 Random Samples)")
print("=" * 80)

# Sample 10 random records
sample_indices = df.sample(10, random_state=42).index

for idx, i in enumerate(sample_indices, 1):
    raw = df.loc[i, 'Content']
    cleaned = df.loc[i, 'cleaned']
    
    print(f"\n[Sample {idx}]")
    print(f"  Raw     : {raw[:90]}...")
    print(f"  Cleaned : {cleaned[:90]}...")
    print(f"  Length  : {len(raw)} → {len(cleaned)} chars")

# ── Cleanup Assessment ─────────────────────────────────────────────────────
print("\n" + "=" * 80)
print("📋 Cleanup Assessment for Sentiment Analysis")
print("=" * 80)
print("""
Reflection Questions:
  ✓ Are sentiment-bearing words preserved? (e.g., "hate", "love", "worst")
  ✓ Are punctuation marks kept? (e.g., "!!!", "??" → important for intensity)
  ✓ Did we lose important negations? (e.g., "can't", "won't" → contractions fixed?)
  ✓ Are there URL remnants, handles, or HTML tags left? (should be gone)
  ✓ Is the text readable and understandable?

"For SENTIMENT ANALYSIS: Emojis and punctuation are PRESERVED as sentiment signals."
If you see issues → adjust preprocess_text() function regex patterns.
""")

# ── Signal vs Noise Summary ────────────────────────────────────────────────
print("\n" + "=" * 80)
print("📊 Signal vs Noise Summary")
print("=" * 80)
chars_removed = (df['Content'].str.len() - df['cleaned'].str.len()).sum()
pct_removed = (chars_removed / df['Content'].str.len().sum()) * 100
print(f"  Total characters removed: {chars_removed:,} ({pct_removed:.1f}%)")
print(f"  Average text reduction  : {(df['Content'].str.len() - df['cleaned'].str.len()).mean():.1f} chars/review")
print(f"\n  → If >50% removed: Check if over-cleaning. Numbers/symbols may be important.")
print(f"  → If <10% removed: Check if under-cleaning. Noise may still be present.")


🔍 Manual Inspection: Raw vs Cleaned (10 Random Samples)

[Sample 1]
  Raw     : u just make everything paid trash app...
  Cleaned : you just make everything paid trash app...
  Length  : 37 → 39 chars

[Sample 2]
  Raw     : I really love this app this app always give me best jazz to Jinja my self...
  Cleaned : i really love this app this app always give me best jazz to jinja my self...
  Length  : 73 → 73 chars

[Sample 3]
  Raw     : Where is hip hop Pretty rude No baby Little baby...
  Cleaned : where is hip hop pretty rude no baby little baby...
  Length  : 48 → 48 chars

[Sample 4]
  Raw     : wrost app ever , every time show there subscribtion plan and didn't play music we'll...
  Cleaned : wrost app ever , every time show there subscribtion plan and did not play music we will...
  Length  : 84 → 87 chars

[Sample 5]
  Raw     : I don't know why people's are gave Less *. But I thought its Like gems of the Music....
  Cleaned : i do not know why people's are gave less *. but i t

# Implementing Tokenization with spaCy

In [25]:
%pip install spacy
import spacy
import subprocess
import sys

# Download the English model
subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])

nlp = spacy.load('en_core_web_sm')
doc = nlp("This app is worst!!!")
for token in doc:
    print(token.text, token.pos_)  # Word + part-of-speech

print("\nSentences:", list(doc.sents))
print("Tokens:", [token.text for token in doc])

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 9.1 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
This DET
app NOUN
is AUX
worst ADJ
! PUNCT
! PUNCT
! PUNCT

Sentences: [This app is worst!!!]
Tokens: ['This', 'app', 'is', 'worst', '!', '!', '!']


In [26]:
import spacy
from spacy.pipeline import EntityRuler

nlp = spacy.load("en_core_web_sm")

# ── Add custom entity ruler for brand names ──────────────────────────────────
if "entity_ruler" not in nlp.pipe_names:
    ruler = nlp.add_pipe("entity_ruler", before="ner")
else:
    ruler = nlp.get_pipe("entity_ruler")

# Define brand names and products to always mark as ORG
patterns = [
    {"label": "ORG", "pattern": "Spotify"},
    {"label": "ORG", "pattern": "spotify"},
    {"label": "ORG", "pattern": "Apple Music"},
    {"label": "ORG", "pattern": "Google Play"},
]

ruler.add_patterns(patterns)

# ── Build capitalization map ────────────────────────────────────────────────
print("=" * 80)
print("🏷️  POS Tagging with Custom NER (Brand Names as ORG)")
print("=" * 80)

capitalization_map = {}
for raw_text in df['Content'].head(500):
    for word in raw_text.split():
        clean_word = word.rstrip(',.!?;:')
        lower_word = clean_word.lower()
        if clean_word and clean_word[0].isupper() and len(clean_word) > 1:
            if lower_word not in capitalization_map:
                capitalization_map[lower_word] = clean_word

def restore_capitalization(text: str) -> str:
    """Restore capitalization before NER processing."""
    words = text.split()
    restored = []
    for word in words:
        clean_word = word.rstrip(',.!?;:')
        punct = word[len(clean_word):]
        if clean_word.lower() in capitalization_map:
            restored.append(capitalization_map[clean_word.lower()] + punct)
        else:
            restored.append(word)
    return ' '.join(restored)

# ── POS Tagging with Custom NER ──────────────────────────────────────────────
print("\nToken Analysis Table (Sample from first review):") 
print(f"{'Token':<15} {'Lemma':<12} {'UPOS':<8} {'Entity Label':<12}")
print("-" * 85)

sample_text = df['cleaned'].iloc[0]
sample_text_restored = restore_capitalization(sample_text)
doc = nlp(sample_text_restored)

# Build entity mapping: token text → entity label
entity_map = {}
for ent in doc.ents:
    for token in ent:
        entity_map[token.text.lower()] = ent.label_

for token in list(doc)[:20]:
    # Show entity label if it's a named entity, otherwise show POS
    if token.text.lower() in entity_map:
        label = entity_map[token.text.lower()]
    else:
        label = token.pos_
    print(f"{token.text:<15} {token.lemma_:<12} {token.pos_:<8} {label:<12}")

# ── Sentence-Level Extraction with Custom NER ───────────────────────────────
print("\n" + "=" * 80)
print("📝 Sentence-Level Analysis (Filtering Named Entities)")
print("=" * 80)

for i, sent in enumerate(list(doc.sents)[:3], 1):
    print(f"\nSentence {i}: {sent.text}")
    
    # Use custom NER to identify brand names
    named_ents_in_sent = {ent.text.lower() for ent in sent.ents}
    
    nouns = [t.text for t in sent if t.pos_ == 'NOUN' and t.text.lower() not in named_ents_in_sent]
    verbs = [t.text for t in sent if t.pos_ == 'VERB' and t.text.lower() not in named_ents_in_sent]
    adjs = [t.text for t in sent if t.pos_ == 'ADJ']
    propns = [ent.text for ent in sent.ents]
    
    if nouns:
        print(f"  📌 Nouns: {nouns}")
    if verbs:
        print(f"  ⚙️  Verbs: {verbs}")
    if adjs:
        print(f"  ✨ Adjectives: {adjs}")
    if propns:
        print(f"  🏢 Named Entities (FILTERED OUT): {propns}")

# ── POS Distribution with Custom NER ─────────────────────────────────────────
print("\n" + "=" * 80)
print("📊 POS Distribution & Sentiment Words (Custom NER)")
print("=" * 80)

pos_counts = {}
lemma_freq = {}

for review in df['cleaned'].head(100):
    review_restored = restore_capitalization(review)
    doc = nlp(review_restored)
    
    # Get named entities (custom ruler takes priority)
    named_ents = {ent.text.lower() for ent in doc.ents}
    
    for token in doc:
        pos_counts[token.pos_] = pos_counts.get(token.pos_, 0) + 1
        
        # Skip named entities like "spotify"
        if token.pos_ in ['ADJ', 'VERB', 'NOUN'] and token.text.lower() not in named_ents:
            lemma_freq[token.lemma_] = lemma_freq.get(token.lemma_, 0) + 1

# Display POS distribution
print("\nPOS Tag Distribution:")
for pos, count in sorted(pos_counts.items(), key=lambda x: x[1], reverse=True):
    pct = (count / sum(pos_counts.values())) * 100
    print(f"  {pos:<8}: {count:>4} ({pct:>5.1f}%)")

# ── Top Sentiment Words ──────────────────────────────────────────────────────
print("\n" + "=" * 80)
print("💡 Top Sentiment-Bearing Words (Spotify ✓ Filtered)")
print("=" * 80)

sentiment_words = {
    k: v for k, v in sorted(lemma_freq.items(), key=lambda x: x[1], reverse=True)
    if len(k) > 2
}

print("\nTop 20 Sentiment Words by Frequency:")
for i, (word, count) in enumerate(list(sentiment_words.items())[:20], 1):
    print(f"  {i:2d}. {word:<15} ({count:>3} occurrences)")

print("\n" + "=" * 80)
print("✅ Success!")
print("=" * 80)
print("""
The custom entity ruler now:
  ✓ Forces "Spotify" → ORG entity (not VERB!)
  ✓ Filters it out from sentiment word counts
  ✓ Handles case variations (spotify, Spotify)
  ✓ Token display shows entity label when applicable
  
You can add more brands to the `patterns` list as needed!
""")

🏷️  POS Tagging with Custom NER (Brand Names as ORG)

Token Analysis Table (Sample from first review):
Token           Lemma        UPOS     Entity Label
-------------------------------------------------------------------------------------
Spotify         spotify      VERB     ORG         
Is              be           AUX      AUX         
very            very         ADV      ADV         
good            good         ADJ      ADJ         
,               ,            PUNCT    PUNCT       
got             get          VERB     VERB        
many            many         ADJ      ADJ         
songs           song         NOUN     NOUN        
And             and          CCONJ    CCONJ       
suggests        suggest      VERB     VERB        
New             new          ADJ      ADJ         
ones            one          NOUN     NOUN        
But             but          CCONJ    CCONJ       
i               I            PRON     PRON        
have            have         VERB     VERB    